## Rayleigh data comparison
This script compares measured Stokes vectors above a Rayleigh atmosphere with varying parameters: tau (optical depth), mu_photon (cos zenith angle), mu_sensor (cos zenith angle), phi (relative azimuth angle between photon emitter and sensor), and albedo of diffuse surface beneath atmosphere. 

In [8]:
import sys
sys.path.append('..')

import json
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ipywidgets import widgets, interact

from parse_rayleigh_data import parse_file

In [9]:
tau_values = [0.02, 0.05, 0.1, 0.15, 0.25, 0.5, 1.0]
albedo_values = [0.0, 0.25, 0.8]
mu_photon_values = [0.1, 0.2, 0.4, 0.6, 0.8, 0.92, 1.0]
mu_sensor_values = [0.02, 0.06, 0.1, 0.16, 0.2, 0.28, 0.32, 0.4, 0.52, 0.64, 0.72, 0.84, 0.92, 0.96, 0.98]
phi_values = [0.0, 30.0, 60.0, 90.0, 120.0, 150.0, 180.0]

In [10]:
ref_prefix = '../rayleigh_data/'

def create_filename(stokes_str, tau):
    tau_str = str(tau) if tau != 1.0 else '1'
    return stokes_str + '_UP_TAU_' + tau_str

data_ref = {'I': {}, 'Q': {}, 'U': {}}

for s in ['I', 'Q', 'U']:
    for t in tau_values:
        filename = create_filename(s, t)
        chunk = parse_file(ref_prefix + filename)
        data_ref[s].update(chunk)

In [13]:
mts_file = '../rayleigh_path_mts.json'

with open(mts_file, 'r') as f:
    data_mts = json.load(f)

In [15]:
import math

def selection_slider(options, value, description):
    return widgets.SelectionSlider(options = options, value = value,description = description, 
                                   continuous_update = False, orientation = 'horizontal', readout = True, 
                                   layout = widgets.Layout(width = '95%'))

stokes_select = widgets.ToggleButtons(options = ['I', 'Q', 'U'], description='stokes', disabled=False,
    tooltips=['Total radiance', '0/90 deg polarized component', '45/135 deg polarized component'],
)

tau_select = selection_slider(options = tau_values, value = 0.5, description = 'tau')
albedo_select = selection_slider(options = albedo_values, value = 0.0, description = 'albedo')
mu_photon_select = selection_slider(options = mu_photon_values, value = 0.6, description = 'mu_photon')
mu_sensor_select = selection_slider(options = mu_sensor_values, value = 0.1, description = 'mu_sensor')
phi_select = selection_slider(options = phi_values, value = 0.0, description = 'phi')

x_select = widgets.ToggleButtons(options = ['tau', 'mu_photon', 'mu_sensor', 'phi'], value = 'phi', 
                                 description = 'x axis', disabled = False)

ui = widgets.VBox(children = [stokes_select, tau_select, albedo_select, mu_photon_select, mu_sensor_select, 
                              phi_select, x_select])

plot_ref = go.Scatter(x = np.array(phi_values), y = np.zeros(len(phi_values)),
                      mode = 'lines',
                      line = dict(), 
                      name = '[Natraj et al. 2009]')

plot_mts = go.Scatter(x = np.array(phi_values), y = np.zeros(len(phi_values)),
                      mode = 'lines', 
                      line = dict(), 
                      name = 'Ours')

layout = go.Layout(
    xaxis_title_text = 'phi',
    yaxis_title_text = 'Stokes value',
    yaxis_range = [0.0, 1.0],
    showlegend = True
)

fig = go.FigureWidget(data = [plot_ref, plot_mts], layout = layout)

def update(stokes_select, tau_select, albedo_select, mu_photon_select, mu_sensor_select, phi_select, x_select):
    if x_select == 'tau':
        x_new = tau_values
        x_title = 'tau'
        data_ref_tmp = data_ref[stokes_select]
        data_ref_new = [data_ref_tmp[t][albedo_select][mu_photon_select][mu_sensor_select][phi_select] for t in tau_values]
        data_mts_tmp = data_mts[stokes_select]
        data_mts_new = [data_mts_tmp[str(t)][str(albedo_select)][str(mu_photon_select)][str(mu_sensor_select)][str(phi_select)] for t in tau_values]
    elif x_select == 'mu_photon':
        x_new = mu_photon_values
        x_title = 'mu_photon'
        data_ref_tmp = data_ref[stokes_select][tau_select][albedo_select]
        data_ref_new = [data_ref_tmp[p][mu_sensor_select][phi_select] for p in mu_photon_values]
        data_mts_tmp = data_mts[stokes_select][str(tau_select)][str(albedo_select)]
        data_mts_new = [data_mts_tmp[str(p)][str(mu_sensor_select)][str(phi_select)] for p in mu_photon_values]
    elif x_select == 'mu_sensor':
        x_new = mu_sensor_values
        x_title = 'mu_sensor'
        data_ref_tmp = data_ref[stokes_select][tau_select][albedo_select][mu_photon_select]
        data_ref_new = [data_ref_tmp[s][phi_select] for s in mu_sensor_values]
        data_mts_tmp = data_mts[stokes_select][str(tau_select)][str(albedo_select)][str(mu_photon_select)]
        data_mts_new = [data_mts_tmp[str(s)][str(phi_select)] for s in mu_sensor_values]
    elif x_select == 'phi':
        x_new = phi_values
        x_title = 'phi'
        data_ref_tmp = data_ref[stokes_select][tau_select][albedo_select][mu_photon_select][mu_sensor_select]     
        data_ref_new = [data_ref_tmp[p] for p in phi_values]
        data_mts_tmp = data_mts[stokes_select][str(tau_select)][str(albedo_select)][str(mu_photon_select)][str(mu_sensor_select)]     
        data_mts_new = [data_mts_tmp[str(p)] for p in phi_values]
    else:
        pass
    
    # Update plots
    with fig.batch_update():
        fig.data[0].x = np.array(x_new)
        fig.data[0].y = np.array(data_ref_new)
        fig.data[1].x = np.array(x_new)
        fig.data[1].y = np.array(data_mts_new)
        
        fig.layout.xaxis.title.text = x_title

out = widgets.interactive_output(update, {'stokes_select': stokes_select, 'tau_select': tau_select, 
                                          'albedo_select': albedo_select, 'mu_photon_select': mu_photon_select, 
                                          'mu_sensor_select': mu_sensor_select, 'phi_select': phi_select,
                                          'x_select': x_select})

display(ui)
fig

FigureWidget({
    'data': [{'mode': 'lines',
              'name': '[Natraj et al. 2009]',
              'typ…